## Secrets

In [0]:
dbutils.secrets.list('scope-mbc')

## Schemas

In [0]:
%sql

create schema if not exists kyndryl.auditoria;
create schema if not exists kyndryl.bronze;
create schema if not exists kyndryl.silver;
create schema if not exists kyndryl.gold;

## Tablas Auditoria

In [0]:
%sql

create or replace table kyndryl.auditoria.parametros(
proceso string,
query_origen string,
path_landing string,
tabla_bronze string,
activo string
);

## Tablas Bronze

In [0]:
%sql

create or replace table kyndryl.bronze.categories(
    category_id string PRIMARY KEY,
    category_name string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.brands(
    brand_id string PRIMARY KEY,
    brand_name string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.products(
    product_id string PRIMARY KEY,
    product_name string,
    brand_id string,
    category_id string,
    model_year string,
    list_price string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.stocks(
    store_id string,
    product_id string,
    quantity string,
    ingestion_timestamp timestamp,
    primary key(store_id, product_id)
);

create or replace table kyndryl.bronze.stores(
    store_id string PRIMARY KEY,
    store_name string,
    phone string,
    email string,
    street string,
    city string,
    state string,
    zip_code string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.staffs(
    staff_id string PRIMARY KEY,
    first_name string,
    last_name string,
    email string,
    phone string,
    active string,
    store_id string,
    manager_id string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.customers(
    customer_id string PRIMARY KEY,
    first_name string,
    last_name string,
    phone string,
    email string,
    street string,
    city string,
    state string,
    zip_code string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.orders(
    order_id string PRIMARY KEY,
    customer_id string,
    order_status string,
    order_date string,
    required_date string,
    shipped_date string,
    store_id string,
    staff_id string,
    ingestion_timestamp timestamp
);

create or replace table kyndryl.bronze.order_items(
    order_id string,
    item_id string,
    product_id string,
    quantity string,
    list_price string,
    discount string,
    ingestion_timestamp timestamp,
    primary key(order_id, item_id)
);

## Tablas Silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.products (
    product_id INT,
    product_name STRING,
    brand_id INT,
    category_id INT,
    model_year SMALLINT,
    list_price DECIMAL(10,2),
    model_year_clean SMALLINT,
    list_price_clean DECIMAL(10,2),
    is_active BOOLEAN,
    processed_at TIMESTAMP,
    source_system STRING
);

CREATE TABLE IF NOT EXISTS silver.orders (
    order_id INT,
    customer_id INT,
    order_status TINYINT,
    order_status_desc STRING,
    order_date DATE,
    required_date DATE,
    shipped_date DATE,
    store_id INT,
    staff_id INT,
    is_on_time BOOLEAN,
    days_to_ship INT,
    processed_at TIMESTAMP,
    source_system STRING
);

CREATE TABLE IF NOT EXISTS silver.order_items (
    order_id INT,
    item_id INT,
    product_id INT,
    quantity INT,
    list_price DECIMAL(10,2),
    discount DECIMAL(4,2),
    total_price DECIMAL(10,2),
    discount_percentage INT,
    processed_at TIMESTAMP,
    source_system STRING,
    PRIMARY KEY (order_id, item_id)
);

## Insert parametros

In [0]:
%sql

delete from kyndryl.auditoria.parametros;

insert into kyndryl.auditoria.parametros
values
("proceso_production",
 "SELECT 
    category_id,
    category_name
FROM 
    production.categories",
  "categories", "bronze.categories","1"),
("proceso_production",
 "SELECT 
    brand_id,
    brand_name
FROM 
    production.brands",
  "brands", "bronze.brands","1"),
("proceso_production",
 "SELECT 
    product_id,
    product_name,
    brand_id,
    category_id,
    model_year,
    list_price
FROM 
    production.products",
  "products", "bronze.products","1"),
("proceso_production",
 "SELECT 
    store_id,
    product_id,
    quantity
FROM 
    production.stocks",
  "stocks", "bronze.stocks","1"),
("proceso_sales",
 "SELECT 
    store_id,
    store_name,
    phone,
    email,
    street,
    city,
    state,
    zip_code
FROM 
    sales.stores",
  "stores", "bronze.stores","1"),
("proceso_sales",
 "SELECT 
    staff_id,
    first_name,
    last_name,
    email,
    phone,
    active,
    store_id,
    manager_id
FROM 
    sales.staffs",
  "staffs", "bronze.staffs","1"),
("proceso_sales",
 "SELECT 
    customer_id,
    first_name,
    last_name,
    phone,
    email,
    street,
    city,
    state,
    zip_code
FROM 
    sales.customers",
  "customers", "bronze.customers","1"),
("proceso_sales",
 "SELECT 
    order_id,
    customer_id,
    order_status,
    order_date,
    required_date,
    shipped_date,
    store_id,
    staff_id
FROM 
    sales.orders",
  "orders", "bronze.orders","1"),
("proceso_sales",
 "SELECT 
    order_id,
    item_id,
    product_id,
    quantity,
    list_price,
    discount
FROM 
    sales.order_items",
  "order_items", "bronze.order_items","1");


SELECT * FROM kyndryl.auditoria.parametros;